In [1]:
!pip install python-docx regex

In [2]:
import docx

from docx import Document
from docx.text.paragraph import Paragraph
from docx.table import Table

from docx.oxml.text.paragraph import CT_P as omxl_paragraph
from docx.oxml.table import CT_Tbl as omxl_table
from docx.oxml.section import CT_SectPr as omxl_section

import regex

In [3]:
def open_docx(path: str) -> Document:
    """
    :params str path: Путь к файлу
    :return Document: Объект docx
    """
    return Document(path)

In [4]:
def get_body(document: Document) -> list[omxl_paragraph | omxl_table | omxl_section]:
    """
    :params Document document: Объект docx
    :return list[omxl_paragraph | omxl_table | omxl_section]: Список объектов находящийся в document
    """
    return [element for element in document.element.body]

In [5]:
def convert_omxl_paragraph_to_text(paragraph: omxl_paragraph, document: Document) -> str:
    """
    :params omxl_paragraph element: параграф oxml
    :params Document document: документ, в котором находится данный параграф
    :return str: текст параграфа
    """
    return Paragraph(paragraph, document).text.lower()

In [6]:
def join_title(paragraphs: list[omxl_paragraph], document: Document) -> str:
    """
    :params list[omxl_paragraph] element: список параграфов omxl
    :params Document document: документ, в котором находится данные параграфы
    :return str: заголовок таблицы
    """

    title: list[str] = list(map(lambda x: convert_omxl_paragraph_to_text(x, document), paragraphs))
    title[0]: str = f'{title[0]}.'
    return ' '.join(title)

In [7]:
def regex_title(text: str) -> bool:
    """
    :params str text: возможный заголовок, где находится таблица N
    :return bool: Присутствует или нет
    """
    return regex.findall(r'таблица\s[0-9]{1,}', text) != []

In [8]:
def convert_text_to_oxml_paragraph(text: str) -> omxl_paragraph:
    """
    :params str text: текст параграфа
    :return omxl_paragraph: параграф omxl
    """

    tmp_document: Document = Document()
    paragraph = tmp_document.add_paragraph(text)
    return paragraph._element

In [9]:
def convert_omxl_table_to_table(element: omxl_table, document: Document) -> Table:
    """
    :params omxl_table element: таблица oxml
    :return Table: объект таблица
    """
    return Table(element, document)

In [10]:
def correct_space(data: list[str]) -> None:
    """
    :params list[str] data: строка таблицы
    :return None: удаляются не нужные пробелы с значениях строки
    """
    for index, value in enumerate(data):
        data[index]: str = ' '.join(value.split())

In [11]:
def correct_title(title: str) -> str:
    return ' '.join(title.split())

In [12]:
def gauss(table: list[list[str]]) -> int | None:
    """
    :params list[list[str]] table: сформированная таблица
    :return int: индекс строки, где значения от 1 до N

    P.S.
        стандарт ЛНД подразумивает после заголовком их нумерацию
        от 1 до N. Т.к. в дальнейшем данная строка не нужна (нет смысла ее обрабатывать),
        следовательно, получаем ее индекс и удаляем.
    """

    for i, row in enumerate(table):
        row_isdigit: list[bool] = list(map(lambda x: x.isdigit(), row))
        if all(row_isdigit):
            length_row: int = len(row)
            if sum(map(int, row)) == (length_row * (length_row + 1)) / 2 and length_row == len(set(row)):
                return i

In [13]:
def all_isdigit(table: list[list[str]]) -> int | None:
    """
    :params list[list[str]] table: сформированная таблица
    :return int: индекс строки, где значения от 1 до N

    P.S.
        В некоторых ЛНД по неизвестным причинам некоторые
        столбцы задваиваются (может быть и не только задваиваются),
        следовательно, как было описано в функции gauss, будем находить
        строку от 1 до N, которая является последней с точки зрения заголовка,
        тем самым, показывающая, как данные считались корректно или нет.
    """

    for i, row in enumerate(table):
        row_isdigit: list[bool] = list(map(lambda x: x.isdigit(), row))
        if all(row_isdigit):
            return i

In [14]:
def find_row_repeat(row: list[str]) -> int | None:
    """
    :params list[str] data: строка таблицы
    :return None: удаляются не нужные пробелы с значениях строки
    """

    for i in range(len(row)):
        if row.count(row[i]) > 1:
            return i

In [15]:
def join_prev_row(layers_table: list[list[str]], join_row: list[list[str]]) -> None:
    """
    ...
    """

    if join_row[0][0] == '':
        for i, element in enumerate(join_row[0]):
            if element:
                layers_table[-1][i] += join_row[0][i]

        join_row.pop(0)

In [16]:
def join_tables(tables: list[Table]) -> list[list[str]]:
    """
    ...
    """

    for i, table in enumerate(tables):
        if i:
            tmp_table = convert_docx_table_to_list_str(table)
            tmp_table = tmp_table[gauss(tmp_table) + 1:]
            join_prev_row(layers_table, tmp_table)
            layers_table += tmp_table

        else:
            layers_table = convert_docx_table_to_list_str(table)

    return layers_table

In [17]:
def convert_docx_table_to_list_str(table: Table | list[Table]) -> list[list[str]]:
    """
    :params Table data: объект Table
    :return list[list[str]]: таблица список строк
    """

    if isinstance(table, list):
        return join_tables(table)

    table: list[list[str]] = [[cell.text for cell in row.cells] for row in table.rows]

    index: int | None = None
    exist_isdigit: int | None = all_isdigit(table)
    if exist_isdigit:
        index: int | None = find_row_repeat(table[exist_isdigit])

    for row in table:
        correct_space(row)
        if index:
            row.pop(index)

    return table

In [18]:
def layers_table(body):
    """
    ...
    """

    index_paragraphs = [i for i in range(len(body)) if isinstance(body[i], omxl_paragraph)]
    new_body = []
    for i in range(len(index_paragraphs) - 1):
        element = body[index_paragraphs[i] + 1:index_paragraphs[i + 1]]

        new_body += [body[index_paragraphs[i]]]
        if len(element) == 1:
            new_body += element

        else:
            new_body += [element]

    new_body += [body[index_paragraphs[-1]]]
    element = body[index_paragraphs[-1] + 1:]
    if len(element) == 1:
        new_body += element

    else:
        new_body += [element]

    return new_body

In [19]:
def new_body(body, document):
    """
    ...
    """

    new_body = []
    index_title_table = None
    unic_index_title = []

    for index, element in enumerate(body):
        if isinstance(element, omxl_paragraph):
            text = convert_omxl_paragraph_to_text(element, document)
            if regex_title (text):
                index_title_table = index

        if isinstance(element, omxl_table):
            table = convert_omxl_table_to_table(element, document)
            table = convert_docx_table_to_list_str(table)


            if 'ТИПОВЫЕ' in table[0][0] or 'РЕГЛАМЕНТ' in table[0][0] or len(table) <= 3:
                continue

            if index_title_table:
                if index_title_table not in unic_index_title:
                    title = join_title(body[index_title_table:index], document)
                    title = convert_text_to_oxml_paragraph(title)
                    new_body += [title]

                unic_index_title += [index_title_table]

            new_body += [element]

    new_body = layers_table(new_body)
    return new_body

In [20]:
"""
Вероятно, что будет не рабочий вариант!!!
"""

def get_tables(body: list[omxl_paragraph | omxl_table | omxl_section], document: Document) -> dict[str: Table]:
    """
    :params list[omxl_paragraph | omxl_table | omxl_section] body: oxml элементы документа
    :params Document document: объект docx
    :return dict[str: Table]: словарь таблиц
    """

    i: int = 0
    tables: dict[str: omxl_table] = {}

    while len(body) > i:
        element: omxl_paragraph | omxl_table = body[i]

        if isinstance(element, omxl_paragraph):
            text: str = convert_omxl_paragraph_to_text(element, document)

            if regex_title(text):
                tmp_body: list[omxl_paragraph | omxl_table] = body[i + 1:]

                for j, el in enumerate(tmp_body):
                    if isinstance(el, omxl_table):
                        title: str = join_title([element] + tmp_body[:j], document)
                        table: Table = convert_omxl_table_to_table(el, document)
                        tables[title]: Table = table

                        # new_body += [convert_text_to_paragraph(title), el]

                        i += j
                        break

            else:
                i += 1

        else:
            i += 1

    return tables

In [21]:
def get_tables(body: list[omxl_paragraph | omxl_table | list[omxl_table]], document: Document) -> dict[str: Table | list[Table]]:
    """
    :params list[omxl_paragraph | omxl_table | list[omxl_table]] body: oxml элементы документа
    :params Document document: объект docx
    :return dict[str: Table]: словарь таблиц
    """

    tables: dict[str: Table | list[Table]] = {}
    for index, element in enumerate(body):
        if isinstance(element, omxl_paragraph):
            title: str = correct_title(convert_omxl_paragraph_to_text(element, document))

        if isinstance(element, omxl_table):
            tables[title] = convert_omxl_table_to_table(element, document)

        elif isinstance(element, list):
            tables[title] = list(map(lambda x: convert_omxl_table_to_table(x, document), element))

    return tables

In [22]:
def name_tables(tables: dict[str: Table]) -> list[str]:
    """
    :params dict[str: Table] tables: словарь таблиц
    :return list[str]: список названия таблиц
    """
    return list(tables.keys())

In [23]:
def count_columns(table: list[list[str]]) -> int:
    """
    :params list[list[str]] table: таблица
    :return int: количество столбцов
    """
    return len(table[gauss(table)])

In [24]:
def count_rows(table: list[list[str]]) -> int:
    """
    :params list[list[str]] table: таблица
    :return int: количество строк
    """
    return len(table)

In [25]:
def get_maxim_length_columns(table: list[list[str]], rows: int, columns: int) -> dict[str: int]:
	"""
    :params list[list[str]] table: таблица
	:params int rows: количество строк
	:params int columns: количество столбцов
    :return dict[str: int]: каждому столбцу максимальная ширина ячейки
    """

	maxim_lenght_columns: dict = dict()

	for i in range(columns):
		maxim_lenght: int = 0
		for j in range(rows):
			maxim_lenght: int = max(maxim_lenght, len(table[j][i]))

		maxim_lenght_columns[f"столбец_{i + 1}"]: int = maxim_lenght + 9 if maxim_lenght % 2 else maxim_lenght + 8

	return maxim_lenght_columns

In [26]:
def format_rows_table(name_table: str, table: list[list[str]], rows: int, columns: int) -> None:
	"""
    :params list[list[str]] table: таблица
	:params int rows: количество строк
	:params int columns: количество столбцов
    :return None: форматирование каждого значения по соот. столбцу
    """

	lenght_columns: dict[str: int] = get_maxim_length_columns(table, rows, columns)

	more_space: int = 0
	if sum(list(lenght_columns.values())) + columns <= len(name_table):
		more_space += len(name_table) // columns

	for i in range(columns):
		format_length: int = lenght_columns[f"столбец_{i + 1}"] + more_space
		for j in range(rows):
			table[j][i]: str = f"{table[j][i]:^{format_length}}"

In [27]:
def sep_values_row(row: list[str]) -> str:
    """
    :params list[str] row: строка
    :return str: соединенная строка разделенная │
    """
    return f"│{'│'.join(row)}│"

In [28]:
def sep_begin_row(index_sep: list[int]) -> str:
    """
    :params list[int] index_sep: индексы разделения (чтобы вставить символ, который разделяет столбцы)
    :return str: разделяющая строка для начальной строки
    """
    return f"├{'┬'.join(list(map(lambda x: '─' * (x - 1), index_sep)))}┤"

In [29]:
def sep_rows(index_sep: list[int]) -> str:
    """
    :params list[int] index_sep: индексы разделения (чтобы вставить символ, который разделяет столбцы)
    :return str: разделяющая строка для строк
    """
    return f"├{'┼'.join(list(map(lambda x: '─' * (x - 1), index_sep)))}┤"

In [30]:
def sep_last_row(index_sep: list[int]) -> str:
    """
    :params list[int] index_sep: индексы разделения (чтобы вставить символ, который разделяет столбцы)
    :return str: разделяющая строка для последней строки
    """
    return f"└{'┴'.join(list(map(lambda x: '─' * (x - 1), index_sep)))}┘"

In [31]:
def sep_title(title: str, lenght: int) -> str:
    """
    :params str title: название таблицы
    :params int lenght: ширина столбцов всего
    :return str: разделение заголовка с названием таблицы
    """
    return f"╭{(lenght - 1) * '─'}╮\n│{title:^{lenght - 1}}│\n"

In [32]:
def index_sep_values(sep_row: str) -> list[int]:
    """
    :params str sep_row: любая строка из таблицы, которая разделена │
    :return list[int]: длина разделителя
    """

    index_sep: list[int] = []

    for i in range(1, len(sep_row)):
        if sep_row[i] == '│':
            index_sep += [i]

    tmp_index_sep: list[int] = [0] * len(index_sep)
    tmp_index_sep[0]: list[int] = index_sep[0]
    for i in range(1, len(index_sep)):
        tmp_index_sep[i]: list[int] = index_sep[i] - index_sep[i - 1]

    return tmp_index_sep

In [33]:
def to_string(name_table: str) -> str:
    """
    :params str name_table: название таблицы
    :return str: сформированная таблица
    """

    table: list[list[str]] = convert_docx_table_to_list_str(tables[name_table])
    rows: int = count_rows(table)
    columns: int = count_columns(table)
    format_rows_table(name_table, table, rows, columns)

    table_string: str = ''
    index_sep: list[int] = index_sep_values(sep_values_row(table[0]))

    for i in range(len(table)):
        if i == len(table) - 1:
            table_string += f"{sep_values_row(table[i])}\n{sep_last_row(index_sep)}"

        else:
            if i:
                table_string += f"{sep_values_row(table[i])}\n{sep_rows(index_sep)}\n"

            else:
                table_string += f"{sep_begin_row(index_sep)}\n{sep_values_row(table[i])}\n{sep_rows(index_sep)}\n"

    return sep_title(name_table, sum(index_sep)) + table_string

In [34]:
document = open_docx('путь_к_файлу.docx')
# document

In [35]:
body = new_body(get_body(document), document)
# body

In [36]:
tables = get_tables(body, document)
# tables

In [37]:
name_tables(tables)

In [38]:
print(to_string('название таблицы из списка выше'))